In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import glob

# Read & Create Data Frames for both data sets

In [2]:
path = ("Results")
allFiles = glob.glob(os.path.join(path, '*.csv'))
allFiles

['Results\\join_data_Mexico_since_2015.csv',
 'Results\\rent_data_Mexico_since_2015.csv',
 'Results\\sell_data_Mexico_since_2015.csv']

In [3]:
for file_ in allFiles:
    df_rent = pd.read_csv (allFiles[1], index_col=None, sep=',', header=0, encoding = 'utf-8')
    df_sell = pd.read_csv (allFiles[2], index_col=None, sep=',', header=0, encoding = 'utf-8')

# Prepare for Merge

In [4]:
#Delete de US currency because as the currency is always fluctuating, it can be srcapped when having the final table
del df_rent['price_aprox_usd']
del df_sell['price_aprox_usd']

#As the two data sets has been worked separately we already know which is for sell and for rent
del df_rent['operation']
del df_sell['operation']

In [5]:
#Delete the id´s because I have find out that the data is a unique data created for a unique composition of data
del df_rent['id']
del df_sell['id']

#Delete the creation date because is irrelevant when joinnin the tables, as they belong to a,
#unique id from a unique composition of data
del df_rent['created_on']
del df_sell['created_on']

In [6]:
df_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258567 entries, 0 to 258566
Data columns (total 8 columns):
property_type          258567 non-null object
place_name             258567 non-null object
state_name             258567 non-null object
lat                    258567 non-null float64
lon                    258567 non-null float64
price                  258567 non-null float64
currency               258567 non-null object
surface_total_in_m2    258567 non-null float64
dtypes: float64(4), object(4)
memory usage: 15.8+ MB


In [7]:
df_sell.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1391580 entries, 0 to 1391579
Data columns (total 8 columns):
property_type          1391580 non-null object
place_name             1391580 non-null object
state_name             1391580 non-null object
lat                    1391580 non-null float64
lon                    1391580 non-null float64
price                  1391580 non-null float64
currency               1391580 non-null object
surface_total_in_m2    1391580 non-null float64
dtypes: float64(4), object(4)
memory usage: 84.9+ MB


# Merge to analysis

In [8]:
pred = pd.merge(
                left=df_rent,
                right=df_sell,
                how='left',
                on=['property_type', 'place_name',
       'state_name', 'lat', 'lon', 'currency','surface_total_in_m2'])

In [9]:
pred.shape

(1051659, 9)

In [10]:
pred = pred.dropna()

In [11]:
pred = pred.drop_duplicates()

In [12]:
pred = pred.rename(columns={'price_y' : 'sell_price', 'price_x' : 'rent_price'})

In [13]:
pred.head()

,property_type,place_name,state_name,lat,lon,rent_price,currency,surface_total_in_m2,sell_price
19,apartment,Quintana Roo,Quintana Roo,21.161908,-86.851528,16500.0,MXN,98.0,1790000.0
23,house,Colima,Colima,19.234996,-103.727212,8500.0,MXN,200.0,2000000.0
25,store,Colima,Colima,19.244782,-103.724963,10000.0,MXN,435.0,1350000.0
40,house,Puebla,Puebla,19.056652,-98.228841,30000.0,MXN,364.0,8140000.0
55,apartment,Puebla,Puebla,19.053259,-98.226545,17000.0,MXN,150.0,3400000.0


In [14]:
pred.to_csv("Results/join_data_Mexico_since_2015.csv", index=False, encoding ='utf-8-sig')

# Convert to JSON

In [16]:
pred.to_json("Results/join_data_Mexico_since_2015.json")